In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import re

In [35]:
#Open firefox and go to url
driver = webdriver.Firefox()

In [36]:
def find_number_class(driver):
    #Get region where the number and class are stored
    class_region = driver.find_element(By.CLASS_NAME, 'col-md-10')
    
    #get the number
    punk_title = class_region.find_element(By.XPATH, './/h1')
    #use regex to extract only the numbers
    numbers_pattern = r'\d+'
    punk_number = re.search(numbers_pattern, punk_title.text)
    
    #get the class
    punk_class = class_region.find_element(By.XPATH, './/h4//a')
    
    return punk_number.group(), punk_class.text

In [37]:
def find_atts(section): #make a long string containing all attributes
    atts = section.find_elements(By.CLASS_NAME, 'col-md-4')
    att_string = ''
    
    for count, element in enumerate(atts):
        
        attribute = element.find_element(By.XPATH, './/a')
        
        if count == 0:
            att_string = att_string + attribute.text
        
        else : 
            att_string = att_string + ',' + attribute.text
    
    return att_string

In [38]:
def find_current_owner(section):
    current_owner_region = section.find_element(By.CLASS_NAME, 'col-md-10')
    
    return current_owner_region.find_element(By.XPATH, './/div//a').text

In [39]:
def punkToFile(punk, fileName):
    
    file = open(fileName, 'a', encoding = 'utf-8')
        
    file.write(punk['number'] + ';' + punk['class'] + ';' + punk['attributes'] + ';'\
                   + punk['current_owner'] + ';' + '\n')
    #If we do not close the file, the file will not be saved
    file.close()

In [40]:
def find_write_transactions(section, punk_number):
    #get the table
    trans_table = section.find_element(By.CLASS_NAME, 'table')
    #get the rows
    trans_rows = trans_table.find_elements(By.XPATH, './/tbody//tr')
    
    file = open(str(punk_number) + '.txt', 'x', encoding = 'utf-8')
    for row in trans_rows:
        info = row.find_elements(By.XPATH, './/td')
        info_string = ''
        
        #get each bit of info
        for count, bit in enumerate(info):
            #if it is the first field, no need to add the separator
            if count == 0:
                info_string = info_string + bit.text
                
            #if it is the fourth field, it is the price
            elif count == 3:
                # if the string is void, the match does not work
                if bit.text != '':
                    #get the price
                    pattern = r'\$\d+,*.*\d*K*M*B*'
                    new_str = re.search(pattern, bit.text).group()
                    
                    #get rid of the last parenthesis and the possible thousand comma
                    number_str = new_str[1:-1].replace(',' , '')
                    
                    #if the string end with a K, multiply by 1000
                    if number_str[-1] == 'K':
                        #change the portion of the string to a float, then multiply
                        #then change back to string
                        number_str = str(float(number_str[:-1]) * 1000)
                    
                    #same, with millions
                    elif number_str[-1] == 'M':
                        number_str = str(float(number_str[:-1]) * 1000000)
                        
                    #same with billions
                    elif number_str[-1] == 'B':
                        number_str = str(float(number_str[:-1]) * 1000000000)
                    
                    #add the separator plus the string
                    info_string = info_string + ';' + number_str
                
                #for empty fields, just add the separator. We need this else,
                # otherwise the separator would not be added
                else:
                    info_string = info_string + ';'
                    
            #the rest of the fields have no special cases
            else :
                info_string = info_string + ';' + bit.text
                
        #write the string, then newline        
        file.write(info_string + '\n')
    file.close()

In [41]:
for number in range(0,6):
    url = 'https://www.larvalabs.com/cryptopunks/details/'
    driver.get(url + str(number))
    punk = {}
    punk['number'], punk['class'] = find_number_class(driver)
    
    #get different sections where info is actually stored
    sections = driver.find_elements(By.CLASS_NAME, 'row')
    
    punk['attributes'] = find_atts(sections[4])
    
    punk['current_owner'] = find_current_owner(sections[6])
    
    punkToFile(punk, 'Punks.txt')
    
    find_write_transactions(sections[7], punk['number'])
    #TODO change date format, get price only in dollars. This should be changed 
    #in the function    